In [2]:
# Install required packages
!pip install transformers streamlit pyngrok --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 57.2 MB/s eta 0:00:00


In [35]:
# ===============================
# Install required packages
# ===============================
!pip install -q transformers streamlit
!npm install -g localtunnel

# ===============================
# Imports
# ===============================
import torch
from pyngrok import ngrok
import logging
import re

# ===============================
# Set up logging
# ===============================
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# ===============================
# Set your Ngrok authtoken here
# ===============================
NGROK_AUTHTOKEN = "YOUR_NGROK_AUTHTOKEN_HERE" ##  NGROK_AUTHTOKEN <<<<<<<<<<==================================================================
!ngrok authtoken {NGROK_AUTHTOKEN}

# ===============================
# Model configuration
# ===============================
MODEL_NAME = "facebook/bart-large-cnn"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ===============================
# Streamlit app code
# ===============================
streamlit_code = f"""
import streamlit as st
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import logging, re

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

MODEL_NAME = "{MODEL_NAME}"
device = "{device}"

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

# Move model to GPU if available
if device == "cuda":
    model = model.to(device)

# Clean input text
def clean_text(text):
    text = re.sub(r'\\s+', ' ', text)
    text = text.strip()
    return text

# Summarization function
def summarize_text(text, max_input_len=1024, max_summary_len=150, min_summary_len=30):
    text = clean_text(text)
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=max_input_len).to(device)
    summary_ids = model.generate(
        inputs["input_ids"],
        max_length=max_summary_len,
        min_length=min_summary_len,
        length_penalty=1.0,
        num_beams=8,
        no_repeat_ngram_size=3,
        early_stopping=True
    )
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Streamlit interface
st.title("Abstractive Text Summarizer (BART)")
st.write("Enter the text you want to summarize:")

user_input = st.text_area("Input text here:", height=300)

if st.button("Generate Summary"):
    if not user_input.strip():
        st.warning("Please enter some text.")
    else:
        with st.spinner("Generating summary..."):
            summary = summarize_text(user_input)
            if len(summary.split()) < 10:
                st.error("Generated summary is too short. Try a longer input.")
                logger.warning("Short summary detected")
            else:
                st.subheader("Summary:")
                st.write(summary)
                logger.info("Summary displayed successfully")
"""

# ===============================
# Save Streamlit app to file
# ===============================
with open("app.py", "w") as f:
    f.write(streamlit_code)

# ===============================
# Expose the app via Ngrok
# ===============================
public_url = ngrok.connect(addr=8501, proto="http")
print(f"Streamlit app URL: {public_url}")

# ===============================
# Run Streamlit app
# ===============================
get_ipython().system_raw("streamlit run app.py &")


⠙⠹⠸⠼⠴⠦⠧⠇
changed 22 packages in 1s
⠇
⠇3 packages are looking for funding
⠇  run `npm fund` for details
⠇Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Streamlit app URL: NgrokTunnel: "https://29f00d55b6cb.ngrok-free.app" -> "http://localhost:8501"
